## <center> Trainner Script 
we are using our custom TrafficSignNet 

In [47]:
from traffic_sign_net import TrafficSignNet # custom Sequential model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report

We will use scikit-image for preprocessing our dataset in lieu of OpenCV as scikit-image provides some additional preprocessing algorithms that OpenCV does not 

In [26]:
from skimage import transform
from skimage import exposure
from skimage import io

In [27]:
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np
import random
import os

initialize the list of data and labels

In [34]:
def load_split(basePath, csvPath):
    data = []
    labels = []
    
    #The format of the data is: Width, Height, X1, Y1, X2, Y2, ClassID, Image Path
    rows = open(csvPath).read().strip().split('\n')[1:] # skipping the first header row
    random.shuffle(rows)
    
    #progressBar
    pbar = tqdm(total=len(rows), desc="loaded:")
    for (i, row) in enumerate(rows):
        (label, imagePath) = row.strip().split(",")[-2:]
        
        imagePath = os.path.sep.join([basePath, imagePath])
        image = io.imread(imagePath)
        
        #images by applying CLAHE
        image = transform.resize(image, (32, 32)) #resize
        image = exposure.equalize_adapthist(image, clip_limit=0.1)
        
        data.append(image)
        labels.append(int(label))
        
        
        pbar.update(1) #progessBAr
    pbar.close()
        
    
    #convert to numpy
    data = np.array(data)
    labels = np.array(labels)
    
    return (data, labels)

In [35]:
# epochs to train for, base learning rate and batch size
EPOCHS = 30
INIT_LR = 1e-3 # 0.001
BS = 64

# load the label names
labelNames = open("signnames.csv").read().strip().split("\n")[1:] #except header
labelNames = [l.split(",")[1] for l in labelNames]
print("numOf Sign Can  be detect ", len(labelNames))

numOf Sign Can  be detect  43


In [36]:
trainPath = r'dataset/Train.csv'
textPath = r'dataset/Test.csv'

print("🔃: loading dataset")
(trainX, trainY) =  load_split("dataset", trainPath)
(testX, testY) = load_split("dataset", textPath)

🔃: loading dataset


loaded::   0%|          | 0/39209 [00:00<?, ?it/s]

loaded::   0%|          | 0/12630 [00:00<?, ?it/s]

In [37]:
print("dataset", trainPath)

dataset dataset/Train.csv


In [38]:
#scale range [0, 1]
trainX = trainX.astype("float32") /255.0
testX = testX.astype("float32") /255.0

In [39]:
#oneHot encode the train & testing labels
numOfLabel = len(np.unique(trainY))
trainY = to_categorical(trainY, numOfLabel)
testY = to_categorical(testY, numOfLabel)

we have significantly more images for some classes than others

In [40]:
# account for skew in the labeled data 
classTotals = trainY.sum(axis=0)
classWeight = classTotals.max() / classTotals

In [41]:
print(len(classTotals),len(classWeight) )

43 43


#### image generator for data augmentation 
 we’re not applying horizontal or vertical flips here as 
traffic signs in the wild will not be flipped.

In [42]:
aug = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode="nearest")

initialize the optimizer and compile the model

In [49]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / (EPOCHS * 0.5))
model = TrafficSignNet.build(width=32, height=32, channel=3, classes=numOfLabel)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

## <center> Train the network
In keras, fit() is much similar to sklearn's fit method, where you pass array of features as x values and target as y values. You pass your whole dataset at once in fit method. Also, use it if you can load whole data into your memory (small dataset).

In fit_generator(), you don't pass the x and y directly, instead they come from a generator. As it is written in keras documentation, generator is used when you want to avoid duplicate data when using multiprocessing. This is for practical purpose, when you have large dataset.
    
#### Warning: THIS FUNCTION IS DEPRECATED.    
It will be removed in a future version. Instructions for updating: Please use Model.fit, which supports generators.

In [52]:
print("trainX shape:", type(trainX), "trainY shape: ", trainY.shape, "\n"
     "testX shape: ", testX.shape, "testY ", testY.shape)

trainX shape: <class 'numpy.ndarray'> trainY shape:  (39209, 43) 
testX shape:  (12630, 32, 32, 3) testY  (12630, 43)


In [51]:
H = model.fit_generator(
    aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY),
    steps_per_epoch=trainX.shape[0] // BS,
    epochs=EPOCHS,
    class_weight=classWeight,
    verbose=1)

I:\ML_Install\anaconda\envs\ObejctDetection\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

evaluate the network & save the model

In [ ]:
predictions = model.predict(testX, batch_size=BS)
print(classification_report(
    testY.argmax(axis=1),
    predictions.argmax(axis=1),
    target_names=labelNames))